Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "05")
cd(ProjDir)

Read the milk data

In [3]:
wd = CSV.read(rel_path("..", "data", "milk.csv"), delim=';')
df = convert(DataFrame, wd);
dcc = filter(row -> !(row[:neocortex_perc] == "NA"), df)
dcc[:kcal_per_g] = convert(Vector{Float64}, dcc[:kcal_per_g])
dcc[:log_mass] = log.(dcc[:mass])

17-element Array{Float64,1}:
  0.6678293725756554
  1.6582280766035324
  1.6808279085207734
  0.9202827531436925
 -0.3856624808119846
 -2.120263536200091 
 -0.7550225842780328
 -1.1394342831883648
  0.4382549309311553
  1.1755733298042381
  2.509599262378372 
  1.6808279085207734
  3.5689691574413787
  4.374876130645041 
  3.70721041079866  
  3.4998353515591547
  4.006423680849631 

Show first 5 rows

In [4]:
first(dcc[[3, 7, 9]], 5)

,kcal_per_g,mass,log_mass
,Float64,Float64⍰,Float64
1,0.49,1.95,0.667829
2,0.47,5.25,1.65823
3,0.56,5.37,1.68083
4,0.89,2.51,0.920283
5,0.92,0.68,-0.385662


Define the Stan language model

In [5]:
m5_6_model = "
data{
    real kcal_per_g[17];
    real log_mass[17];
}
parameters{
    real a;
    real bm;
    real sigma;
}
model{
    vector[17] mu;
    sigma ~ uniform( 0 , 1 );
    bm ~ normal( 0 , 1 );
    a ~ normal( 0 , 100 );
    for ( i in 1:17 ) {
        mu[i] = a + bm * log_mass[i];
    }
    kcal_per_g ~ normal( mu , sigma );
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [6]:
stanmodel = Stanmodel(name="m5_5_model",
monitors = ["a", "bm", "sigma"],
 model=m5_6_model, output_format=:mcmcchain);
# Input data for cmdstan
m5_6_data = Dict("N" => size(dcc, 1),
  "kcal_per_g" => dcc[:kcal_per_g],
  "log_mass" => dcc[:log_mass]);
# Sample using cmdstan
rc, chn, cnames = stan(stanmodel, m5_6_data, ProjDir, diagnostics=false,
  CmdStanDir=CMDSTAN_HOME);
# Rethinking results
rethinking_results = "
        mean    sd   5.5%  94.5% n_eff  Rhat
        mean   sd  5.5% 94.5% n_eff Rhat
 a      0.70 0.06  0.61  0.79   926 1.00
 bm    -0.03 0.02 -0.07  0.01   986 1.00
 sigma  0.18 0.04  0.13  0.25   628 1.01
";


File /Users/rob/.julia/dev/StatisticalRethinking/scripts/05/tmp/m5_5_model.stan will be updated.

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is -0.432502, but must be > 0!  (in '/Users/rob/.julia/dev/StatisticalRethinking/scripts/05/tmp/m5_5_model.stan' at line 18)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is -0.049458, but must be > 0!  (in '/Users/rob/.julia/dev/StatisticalRethinking/scripts/05/tmp/m5_5_model.stan' at line 18)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrice

Describe the draws

In [7]:
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean          SD        Naive SE       MCSE      ESS
    a  0.704458859 0.058900286 0.00093129530 0.0014315622 1000
   bm -0.031615675 0.024470935 0.00038691946 0.0005780587 1000
sigma  0.184539657 0.039175545 0.00061941975 0.0009990761 1000

Quantiles:
          2.5%        25.0%       50.0%       75.0%       97.5%   
    a  0.58746805  0.666580250  0.7050610  0.741550500 0.820761550
   bm -0.08028639 -0.047507575 -0.0317967 -0.015903075 0.016978703
sigma  0.12544568  0.157053750  0.1783535  0.205513250 0.278520300



End of `05/5.5s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*